In [ ]:
import matplotlib.pyplot as plt
# from matplotlib.path import Path
# https://matplotlib.org/stable/api/path_api.html
# from matplotlib.patches import PathPatch

import numpy as np

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry import LinearRing
import shapely
from shapely import affinity

from BFS_erebus_oh import Obstacle, Project2Map

import math

from tqdm import tqdm

map = Project2Map()

# o = Obstacle(xs, ys, clearance=2)
# o = LinearRing(pts)

# print(o.is_closed)

# test = [20, 18.5]
test = [30, 10]
# ax.fill(*o.xy)

# ax.plot(test[0], test[1], marker='o', color='pink')

# o.draw(ax)

# letter_e.draw(ax, color='blue')

# # ax.fill(*letter_e.plot_points(), facecolor='blue')

# ax.plot(test[0], test[1], marker='o', color='pink')

# letter_e.collides(test)
# p = Point(test)


# whether points is +/- 2 mm from boundary
# print(o.dwithin(p, 2))
# # whether point is inside
# ßprint(Polygon(o).contains(p))

map.map(0, 0)

Generating Map: 100%|██████████| 180/180 [00:00<00:00, 216.45it/s]

Saved map as map.png
indexing: 49, 0


array([0., 0., 0., 1.], dtype=float32)

In [2]:
map.map(179, 49)

indexing: 0, 179


array([0., 0., 0., 1.], dtype=float32)

In [4]:
image[0,0]

array([0., 0., 0., 1.], dtype=float32)